# SQL in Python Assignment

In [12]:
import pandas as pd
from sqlalchemy import *


ModuleNotFoundError: No module named 'psycopg2'

### Connect to the personal database you created yesterday.
Remember the way the credentials work:
'''
postgres_user = 'dsbc_student'

postgres_pw = '7*.8G9QH21'

postgres_host = '142.93.121.174'

postgres_port = '5432'

postgres_db = 'dvdrentals'
'''

In [22]:
user = 'dsbc_student'
pw = '7*.8G9QH21'
host = '142.93.121.174'
port = '5432'
db = 'fueleconomy'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    user, pw, host, port, db))

### Obtain a list of tables that are in the database.

In [23]:
engine.table_names()

['vehicles', 'common']

### Obtain a list of all the fields that are in the vehicles table along with the data type of each.

In [24]:
meta = MetaData()
vehicles = Table('vehicles', meta, autoload=True, autoload_with=engine)
fields = [(c.name, c.type) for c in vehicles.columns]
fields_df = pd.DataFrame(fields, columns =['Name', 'Type'])
fields_df

,Name,Type
0,id,INTEGER
1,make,VARCHAR
2,model,VARCHAR
3,year,INTEGER
4,class,VARCHAR
5,trans,VARCHAR
6,drive,VARCHAR
7,cyl,SMALLINT
8,displ,REAL
9,fuel,VARCHAR


### Write a SELECT query to retrieve all records from the database and load them into a Pandas data frame.

In [25]:
sql = """
SELECT * FROM vehicles;
"""
results = pd.read_sql_query(sql, con=engine, index_col='id')
results

,make,model,year,class,trans,drive,cyl,displ,fuel,hwy,cty
id,,,,,,,,,,,
27550,AM General,DJ Po Vehicle 2WD,1984,Special Purpose Vehicle 2WD,Automatic 3-spd,2-Wheel Drive,4.0,2.5,Regular,17,18
28426,AM General,DJ Po Vehicle 2WD,1984,Special Purpose Vehicle 2WD,Automatic 3-spd,2-Wheel Drive,4.0,2.5,Regular,17,18
27549,AM General,FJ8c Post Office,1984,Special Purpose Vehicle 2WD,Automatic 3-spd,2-Wheel Drive,6.0,4.2,Regular,13,13
28425,AM General,FJ8c Post Office,1984,Special Purpose Vehicle 2WD,Automatic 3-spd,2-Wheel Drive,6.0,4.2,Regular,13,13
1032,AM General,Post Office DJ5 2WD,1985,Special Purpose Vehicle 2WD,Automatic 3-spd,Rear-Wheel Drive,4.0,2.5,Regular,17,16
...,...,...,...,...,...,...,...,...,...,...,...
33305,smart,fortwo electric drive convertible,2013,Two Seaters,Automatic (A1),Rear-Wheel Drive,NaN,NaN,Electricity,93,122
34393,smart,fortwo electric drive convertible,2014,Two Seaters,Automatic (A1),Rear-Wheel Drive,NaN,NaN,Electricity,93,122
31065,smart,fortwo electric drive coupe,2011,Two Seaters,Automatic (A1),Rear-Wheel Drive,NaN,NaN,Electricity,79,94


### Create a table that contains all Audi vehicles with a city fuel efficiency greater than 18.

In [26]:
#issue with creating my own database on SQL so just wrote thw code

sql = """
CREATE TABLE IF NOT EXISTS audi AS
SELECT * FROM vehicles
WHERE make = 'Audi' AND cty > 18;
"""
results = pd.read_sql_query(sql, con=engine, index_col='id')
results

ProgrammingError: (psycopg2.errors.InsufficientPrivilege) permission denied for schema public

[SQL: 
CREATE TABLE IF NOT EXISTS audi AS
SELECT * FROM vehicles
WHERE make = 'Audi' AND cty > 18;
]
(Background on this error at: http://sqlalche.me/e/13/f405)

### Delete all records from the temp table you just created.

In [28]:
#again permission issues

delete = "DELETE FROM audi;"

engine.execute(delete)

ProgrammingError: (psycopg2.errors.UndefinedTable) relation "audi" does not exist
LINE 1: DELETE FROM audi;
                    ^

[SQL: DELETE FROM audi;]
(Background on this error at: http://sqlalche.me/e/13/f405)

### Insert records from the vehicles table back into the table you created where the make is Audi and the city fuel efficiency is greater than 20.

In [29]:
#manually wrote code

insert = """
INSERT INTO audi
SELECT * from vehicles
WHERE make='audi' and cty > 20;
"""

engine.execute(insert)

ProgrammingError: (psycopg2.errors.UndefinedTable) relation "audi" does not exist
LINE 2: INSERT INTO audi
                    ^

[SQL: 
INSERT INTO audi
SELECT * from vehicles
WHERE make='audi' and cty > 20;
]
(Background on this error at: http://sqlalche.me/e/13/f405)

### Update the drive field of the records in the table you created to change any "4-Wheel or All-Wheel Drive" values to "All-Wheel Drive."

In [31]:
update = """
UPDATE audi
set drive = 'All-Wheel Drive'
Where drive = '4-Wheel or All-Wheel Drive'
"""

engine.execute(update)

ProgrammingError: (psycopg2.errors.UndefinedTable) relation "audi" does not exist
LINE 2: UPDATE audi
               ^

[SQL: 
UPDATE audi
set drive = 'All-Wheel Drive'
Where drive = '4-Wheel or All-Wheel Drive'
]
(Background on this error at: http://sqlalche.me/e/13/f405)

### Write a for loop that iterates over the list of makes provided below, selects the top 10 vehicles with the highest city fuel efficiency for each make, and then concatenates all the results together into a single Pandas data frame.

In [33]:
makes = ['Ford', 'Chevrolet', 'Toyota', 'Honda']

In [34]:
topCars = pd.DataFrame()
topCarSQLFront = "SELECT * from vehicles WHERE make = '"
topCarSQLEnd = "' ORDER BY cty DESC LIMIT 10"
for x in makes:
    sql = topCarSQLFront + x + topCarSQLEnd
    results = pd.read_sql_query(sql, con=engine, index_col='id')
    topCars = topCars.append(results)
topCars

,make,model,year,class,trans,drive,cyl,displ,fuel,hwy,cty
id,,,,,,,,,,,
33024,Ford,Focus Electric,2013,Compact Cars,Automatic (A1),Front-Wheel Drive,NaN,NaN,Electricity,99,110
34130,Ford,Focus Electric,2014,Compact Cars,Automatic (A1),Front-Wheel Drive,NaN,NaN,Electricity,99,110
32278,Ford,Focus BEV FWD,2012,Compact Cars,Automatic (variable gear ratios),Front-Wheel Drive,NaN,NaN,Electricity,99,110
30965,Ford,Ranger Pickup 2WD,2001,Standard Pickup Trucks 2WD,Automatic (A1),2-Wheel Drive,NaN,NaN,Electricity,54,62
30966,Ford,Ranger Pickup 2WD,2000,Standard Pickup Trucks 2WD,Automatic (A1),2-Wheel Drive,NaN,NaN,Electricity,54,62
30971,Ford,Ranger Pickup 2WD,1999,Standard Pickup Trucks 2WD,Automatic (A1),2-Wheel Drive,NaN,NaN,Electricity,54,62
30967,Ford,Ranger Pickup 2WD,2000,Standard Pickup Trucks 2WD,Automatic (A1),2-Wheel Drive,NaN,NaN,Electricity,52,59
30972,Ford,Ranger Pickup 2WD,1999,Standard Pickup Trucks 2WD,Automatic (A1),2-Wheel Drive,NaN,NaN,Electricity,44,50
33083,Ford,Fusion Hybrid FWD,2013,Midsize Cars,Automatic (variable gear ratios),Front-Wheel Drive,4.0,2.0,Regular,47,47
